# to generate illumina nex and ht sample sheet with barcodes

In [8]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from io import StringIO
import os
import csv
import sys
import glob
from collections import OrderedDict,defaultdict,Counter
import datetime
%matplotlib inline 

import xlsxwriter
from scipy import stats

import seaborn as sns
# from pandas.tools import plotting
from matplotlib import pyplot as plt


from Bio.Alphabet import generic_dna,generic_rna,generic_protein
import primer3
from Bio import SeqIO
from Bio.Seq import Seq
from Bio import Entrez
import math
import pysam



In [2]:
 !pip3 install primer3-py

In [2]:
current_folder_path, current_folder_name = os.path.split(os.getcwd())


In [3]:
# define the name of the directory to be created
samples_folder = current_folder_path+'/samples'

try:  
    os.mkdir(samples_folder)
except OSError:  
    print ("Creation of the directory %s failed" % samples_folder)
else:  
    print ("Successfully created the directory %s " % samples_folder)

Successfully created the directory /Users/ltao/Projects/glioma/exp/190501/samples 


In [5]:
# try:  
#     os.rmdir(path)
# except OSError:  
#     print ("Deletion of the directory %s failed" % path)
# else:  
#     print ("Successfully deleted the directory %s" % path)

In [6]:
input_key=['Sample_ID',
  'Sample_Name',
  'Sample_Plate',
  'Sample_Well',
  'I7_Index_ID',
  'index',
  'I5_Index_ID',
  'index2',
  'Sample_Project',
  'Description']

In [18]:
## Setting Sample Sheet
exp_name='mipsa'
exp_date= str(datetime.datetime.today().date())

In [19]:
samplesheet_file='/Users/ltao/Projects/Data/miseq/nex/MS8001943-300V2.csv'
sample_sheet = pd.read_csv(samplesheet_file)
input_key = sample_sheet.iloc[[19]].values.tolist()
sample_sheet.columns=input_key
sample_sheet=sample_sheet.drop([20])
sample_sheet.loc[2,'Sample_Name']=exp_name
sample_sheet.loc[3,'Sample_Name']=exp_date

sample_sheet

,Sample_ID,Sample_Name,Sample_Plate,Sample_Well,I7_Index_ID,index,I5_Index_ID,index2,Sample_Project,Description
0,IEMFileVersion,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Investigator Name,Liming,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Experiment Name,mipsa,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Date,2019-05-17,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Workflow,GenerateFASTQ,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Application,FASTQ Only,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Assay,Nextera,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Description,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Chemistry,Amplicon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
sheets = pd.read_excel('/Users/ltao/Projects/Data/miseq/nex/Nextra-Ads.xlsx', sheet_name=None)
nexads=dict()
for sheet_name, sheet in sheets.items():
    print (sheet_name)
nex_i5=sheets["plate AD1-i5"]
nex_i5.set_index('96 Well Columns', inplace=True)
nex_i7=sheets["plate Ad2 i7"]
nex_i7.set_index('96 Well Columns', inplace=True)
layout_i5=sheets['layout_i5']
layout_i7=sheets['layout_i7']

plate AD1-i5
plate Ad2 i7
layout_i5
layout_i7
Fill Along Columns
Sheet1


In [8]:
def primer_loc(rows,cols):
    locs=list()
    for r5 in rows:
        for c5 in cols:
             locs.append(str(r5)+str(c5))
    return locs

In [9]:
### combine i5, i7 by rows: A:A; B:B....H:H
def xbarcodes_pairs(rows_i5='ABCD',rows_i7='ABCD',cols_i5='5678',cols_i7='5678'):
    barcodes_i5=primer_loc(rows_i5,cols_i5)
    barcodes_i7=primer_loc(rows_i7,cols_i7)
    xbarcodes_i5=list()
    for j in barcodes_i5:
        for i in range(len(cols_i7)):
            xbarcodes_i5.append(j)
    barcodes_pair=list(zip(xbarcodes_i5,barcodes_i7))
    return barcodes_pair
#     return xbarcodes_i5

In [11]:

# non-dig-hcc38-0.7xbeads:

# Ad1: col 11
# Ad2:1-6


rows_i5='ABCDEFGH'
rows_i7='ABCDEFGH'
cols_i5=[4,]
cols_i7=[1,2]

idh=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

cols_i5=[4,]
cols_i7=[3,4]
# cols_i7=[7,8,9,10,11,12]

g_5x=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

cols_i5=[4,]
cols_i7=[5,6]

g_17x_a=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

cols_i5=[5,]
cols_i7=[7,]

g_17x_b=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)
g_17x =g_17x_b+g_17x_a

cols_i5=[5,]
cols_i7=[8]
hcc38_g_17x_q5=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)

cols_i5=[5,]
cols_i7=[9,10]

g_17x_q5=xbarcodes_pairs(rows_i5 ,rows_i7 ,cols_i5 ,cols_i7)



In [12]:
nex_i5_dic=nex_i5.to_dict(orient="index")
nex_i7_dic=nex_i7.to_dict(orient="index")

In [13]:
input_bp=OrderedDict()


In [14]:
### chose runtype
color = int(input("NovaSeq, MiSeq, HiSeq 2000/2500 == four_color, \n MiniSeq, NextSeq, HiSeq 3000/4000 == two_color, \n what is your run? \n pls input 2 or 4\n"))

if color == 4:
    runtype ="four_color"
    print ("your run is ###four_color###")
elif color == 2:
    runtype ="two_color"
    print ("your run is ###two_color###")
else:
    print ('pls define your runtype, 2 or 4')

NovaSeq, MiSeq, HiSeq 2000/2500 == four_color, 
 MiniSeq, NextSeq, HiSeq 3000/4000 == two_color, 
 what is your run? 
 pls input 2 or 4
4
your run is ###four_color###


In [15]:
## for NovaSeq, MiSeq, HiSeq 2000/2500, 
for plate,bps in {"idh":idh,"g_5x":g_5x,"g_17x":g_17x,"hcc38_pbs":hcc38_g_17x_q5,"g_17x_q5":g_17x_q5}.items():
    for bp in bps:
        samples_input_dict=OrderedDict()
        if runtype == "four_color":
            i5_barcodes = nex_i5_dic[bp[0]]['i5 Bases for Sample Sheet NovaSeq, MiSeq, HiSeq 2000/2500\xa0']
        else:
            i5_barcodes = nex_i5_dic[bp[0]]['i5 Bases for Sample Sheet MiniSeq, NextSeq, HiSeq 3000/4000 ']

        i7_barcodes = nex_i7_dic[bp[1]]['on Sample Sheet']
        for k in input_key:
            samples_input_dict[k] = 'i7_'+bp[1]+'_i5_'+bp[0]
        samples_input_dict['index2'] =i5_barcodes
        samples_input_dict['index'] =i7_barcodes
        samples_input_dict['Sample_Plate'] = plate
        input_bp['i7_'+bp[1]+'_i5_'+bp[0]]=samples_input_dict


In [ ]:
# barcodes_file=samples_folder+'/barcodes.xlsx'

# input_bp_df=pd.DataFrame.from_dict(input_bp, orient='index')
# input_bp_df.to_excel(barcodes_file)

In [16]:
runsample_sheet=samples_folder+'/runsample_sheet.csv'

input_bp_df=pd.DataFrame.from_dict(input_bp, orient='index')
result = sample_sheet.append(input_bp_df)
result.to_csv(runsample_sheet,header=False, index=False)

In [17]:
fastqs_folder=current_folder_path+'/fastqs/'
os.mkdir(fastqs_folder)

### Prepare all fastqs by shell in merge_script


In [17]:
# fastqs_folder=current_folder_path+'/fastqs/'
# os.mkdir(fastqs_folder)
merge_next_file=fastqs_folder+"merge_nex.txt"
# prepare merge config files
fastqs=list()
for x in os.listdir(fastqs_folder):
    if 'fastq' in x:
        fastqs.append(x)
fastqs=sorted(fastqs
             )
merge_key=['sample','r1','r2']
merge_nex=OrderedDict()

for x in zip(fastqs[::2],fastqs[1::2]):
    pe_reads=dict()
    sindex=x[0].index('_S')
    name=x[0][:sindex]
#     print (name)
    mk=input_bp[name]["Sample_Plate"]+"_"+name
    pe_reads['sample']=mk
    pe_reads ['r1']=x[0]
    pe_reads['r2']=x[1]
    merge_nex[mk]=pe_reads
df_merge_nex=pd.DataFrame.from_dict(merge_nex, orient='index')
df_merge_nex.to_csv(merge_next_file,sep="\t")  
# merge_nex

In [4]:
fastqs_folder=current_folder_path+'/fastqs/'


### Go to merge shell script 

# allele dropout analysis

In [5]:
os.chdir(fastqs_folder)

In [19]:

mfasta = [f for f in os.listdir(fastqs_folder) if f.endswith('MERGED')]
mfasta=sorted(mfasta, reverse=True)

In [20]:
def sum_fastq(mf):
    records = list(SeqIO.parse(mf, "fasta"))
    total_reads=len(records)    
    tmp=list()
    for rcd in records:
        tmp.append(len(rcd))
    tmp=sorted(tmp)
    if total_reads>0:
#         print (total_reads,tmp[0], tmp[-1])
        min_len=tmp[0]
        max_len=tmp[-1]
#         print (min_len,max_len )
    else:
        print ('no reads')
        min_len='no reads'
        max_len='no reads'

    return(total_reads, min_len, max_len)

In [21]:
def rank_amplicon(merged_fasta):

    records = list(SeqIO.parse(merged_fasta, "fasta"))
    s=list()

    for rcd in records:
        s.append((str(rcd.seq.lower()),rcd.id))
    
    unique_reads = defaultdict(list)
    for k, v in s:
        unique_reads[k].append(v)
    rst=dict()
    rst['total_reads']=len(records)

    for k,v in unique_reads.items():
        rst[k]=len(v)
#     return rst
    return unique_reads

In [22]:
counts=dict()
for mfa in mfasta:
    tmp=dict()
    tmp['ranking']= rank_amplicon(mfa)
    tmp['sum']=sum_fastq(mfa)
    counts[mfa]=tmp

no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads
no reads


In [6]:
import os
from sys import argv, exit
import pysam
from collections import Counter

# CHROM = "chr3"
# POS = 168818653

# if len(argv) != 3:
# 	print("Usage: python calc_crispr_efficiency.py input_path output.tsv")
# 	exit(-1)

def parse_cigar(cigartuples):
	has_D = False
	has_I = False
	has_clip = False

	for cbase, clen in cigartuples:
		has_D = has_D or (cbase == 2)
		has_I = has_I or (cbase == 1)
		has_clip = has_clip or (cbase == 4 or cbase == 5)

	return 2 if has_clip else (1 if has_D or has_I else 0)




In [ ]:
# with open(argv[2], "w") as fout:
# 	fout.write("Cell\tEfficiency\tValid Reads\n")
# 	for file in os.listdir(os.path.abspath(argv[1])):
# 		if file.endswith(".bam"):
# 			cell = '_'.join(file.split('_')[1:-1])
# 			numer = denom = 0
# 			with pysam.AlignmentFile(file) as sam_in:
# 				cigardict = Counter()
# 				for aread in sam_in:
# 					if not aread.is_unmapped and aread.is_reverse and aread.reference_name == CHROM and aread.reference_start + 1 == POS:
# 						value = parse_cigar(aread.cigartuples)
# 						numer += value == 1
# 						denom += value < 2
# 						if value < 2:
# 							cigardict[aread.cigarstring] += 1
# 			print(cell)
# 			print(cigardict)
# 			print()
# 			fout.write("{}\t{:.0%}\t{}\n".format(cell, numer * 1.0 / denom, denom))


## get gene name

In [7]:
refGenetxt="/Users/ltao/ref/human/hg19/refGene.txt"


In [8]:
df=pd.read_csv(refGenetxt,sep='\t',header=None)

In [10]:
col_name=['bin',
 'name',
 'chrom',
 'strand',
 'txStart',
 'txEnd',
 'cdsStart',
 'cdsEnd',
 'exonCount',
 'exonStarts',
 'exonEnds',
 'score',
 'name2',
 'cdsStartStat',
 'cdsEndStat',
 'exonFrames']


In [11]:
df.columns = col_name

In [12]:
refGene=df.to_dict(orient='index')

In [27]:

def get_gene_name(chr_number,start_p,end_p):
    tmp={'bin': '',
 'name': '',
 'chrom': '',
 'strand': '',
 'txStart': '',
 'txEnd': '',
 'cdsStart': '',
 'cdsEnd': '',
 'exonCount': '',
 'exonStarts': '',
 'exonEnds': '',
 'score': '',
 'name2': '',
 'cdsStartStat': '',
 'cdsEndStat': '',
 'exonFrames': ''}
    gene_names=list()
    for k,v in refGene.items():
        ref_id=v['name']
        chrom=v['chrom']
        strand=v['strand']
        txStart=v['txStart']
        txEnd=v['txEnd']
#     'cdsStart': 794826,
#     'cdsEnd': 794826,
#     'exonCount': 6,
#     'exonStarts': '762970,764382,783033,787306,788050,788770,',
#     'exonEnds': '763155,764484,783186,787490,788146,794826,',
#     'score': 0,
        gene_name=v['name2']
#     'cdsStartStat': 'unk',
#     'cdsEndStat': 'unk',
#     'exonFrames': '-1,-1,-1,-1,-1,-1,'
        if chrom=='chr'+ str(chr_number) and txEnd >= start_p >= txStart and txStart <=end_p<=txEnd:
            gene_names.append(v)
    if len(gene_names) == 0:
        gene_names.append(tmp)
    return gene_names

In [25]:
# # s=0
# for k, v in counts.items():
#     if not 'mgh'==k[:3]:
#         total_reads=v['sum'][0]
# #         print (total_reads)
#         bam= k +".sorted.bam"
#         print (bam)
#         with pysam.AlignmentFile(bam) as sam_in:
# #             print (sam_in.count(chr_number,start_p,end_p))
#             for aread in sam_in:
#                 chr_number=aread.reference_name
#                 start_p=aread.reference_start
#                 end_p=aread.reference_end
#                 v=get_gene_name(chr_number,start_p,end_p) # v is a list of dict
#                 gene=v[0]['name2']
# #                 print (aread.is_unmapped, aread.is_reverse, aread.reference_name,aread.reference_start, aread.reference_end, aread.mapping_quality, gene)
# #                 print (aread.reference_name,aread.reference_start, aread.reference_end,aread.get_reference_sequence())
#                 print (aread.reference_name,aread.reference_start, aread.reference_end,aread.cigarstring, aread.mapping_quality, gene)
#             print (sam_in.count())
# #                 print (aread.cigartuples, aread.mapping_quality, gene)
                
# # 				if not aread.is_unmapped and aread.is_reverse and aread.reference_name == CHROM and aread.reference_start + 1 == POS:
# # 					value = parse_cigar(aread.cigartuples)
# # 					numer += value == 1
# # 					denom += value < 2
# # 					if value < 2:
# # 						cigardict[aread.cigarstring] += 1
# # 		print(cell)
# # 		print(cigardict)
# # 		print()        

# #         s+=1
# # print (s)

hcc38_rtl_non_dig_i7_H3_i5_H11_MERGED.sorted.bam
2 3482520 3482717 197M 60 TRAPPC12
2 3482520 3482717 197M 60 TRAPPC12
2 3482520 3482717 197M 60 TRAPPC12
2 3482520 3482717 197M 60 TRAPPC12
2 3482520 3482717 197M 60 TRAPPC12
2 3482520 3482717 14M1I183M 60 TRAPPC12
2 3482520 3482717 197M 60 TRAPPC12
2 3482520 3482717 197M 60 TRAPPC12
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 24M1I157M 60 IDH1
2 209113024 209113205 24M1I157M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 24M1D156M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 24M1I157M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 2091

15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 211M1I16M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
73
hcc38_rtl_non_dig_i7_H1_i5_H11_MERGED.sorted.bam
2 209113024 209113205 16M1D7M1D156M 60 IDH1
1
hcc38_rtl_non_dig_i7_G3_i5_G11_MERGED.sorted.bam
2 3482520 3482717 197M 60 TRAPPC12
2 3482520 3482717 197M 60 TRAPPC12
2 3482520 3482717 197M 60 TRAPPC12
2 3482520 3482717 197M 60 TRAPPC12
2 3482520 3482717 197M 60 TRAPPC12
2 3482520 3482717 197M 60 TRAPPC12
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 24M1I157M 60 IDH1
2 209113024 209113205 24M1D156M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 24M1D156M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 20911320

2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 24M1D156M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 24M1D156M 60 IDH1
2 209113024 209113205 168M1D12M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 24M1D156M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 24M1D156M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 24M1I157M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181M 60 IDH1
2 209113024 209113205 181

15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 175M1D51M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 175M1D51M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 217M1D9M 60 TP53BP1
15 43748010 43748237 175M1D51M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 43748237 227M 60 TP53BP1
15 43748010 4374823

X 102828883 102829037 40M2D112M 60 
X 102828883 102829037 40M2D112M 60 
X 102828883 102829037 40M2D112M 60 
X 102828883 102829037 40M2D112M 60 
X 102828883 102829037 40M2D112M 60 
X 102828883 102829037 40M2D112M 60 
X 102828883 102829037 40M2D112M 60 
X 102828883 102829037 40M2D112M 60 
X 102828883 102829037 40M2D112M 60 
X 102828883 102829037 40M2D112M 60 
X 102828883 102829037 40M2D112M 60 
X 109092754 109092913 97M2D60M 60 
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330435 94M4D69M2S 60 PAK3
X 110330268 110330432 94M4D66M5S 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M6D69M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M6D69M 60 PAK3
X 110330268 110330437 94M6D69M 60 PAK3
X 110330268 110330437 94M6D69M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M6D69M 60 PAK3
X 110330268 110330436 94M4D70M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 1103304

X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M6D69M 60 PAK3
X 110330268 110330437 94M2D73M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 4M1D89M6D69M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M6D69M 60 PAK3
X 110330268 110330437 94M6D69M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M2D73M 60 PAK3
X 110330268 110330437 94M2D73M 60 PAK3
X 110330268 110330437 94M6D69M 60 PAK3
X 110330268 110330437 94M6D69M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M2D73M 60 PAK3
X 110330268 110330437 94M4D71M 60 PAK3
X 110330268 110330437 94M6D69M 60 PAK3
X 110330268 110330437

X 111187472 111187631 159M 60 TRPC5
X 111187472 111187631 23M2I136M 60 TRPC5
X 111187472 111187631 159M 60 TRPC5
X 111187472 111187631 23M4D132M 60 TRPC5
X 111187472 111187631 59M2D98M 60 TRPC5
X 111187472 111187631 59M4D96M 60 TRPC5
X 111187472 111187631 59M2D98M 60 TRPC5
X 111187472 111187631 59M6D94M 60 TRPC5
X 111187472 111187631 59M2D98M 60 TRPC5
X 111187472 111187631 59M2D98M 60 TRPC5
X 111187472 111187631 57M4D98M 60 TRPC5
X 111187472 111187631 159M 60 TRPC5
X 111187472 111187631 23M2I136M 60 TRPC5
X 111187472 111187631 59M4D96M 60 TRPC5
X 111187472 111187631 59M2D98M 60 TRPC5
X 111187472 111187631 59M4D96M 60 TRPC5
X 111187472 111187631 14M1D144M 60 TRPC5
X 111187472 111187631 59M2D98M 60 TRPC5
X 111187472 111187631 59M2D98M 60 TRPC5
X 111187472 111187631 57M6D96M 60 TRPC5
X 111187472 111187631 11M1I44M6D98M 60 TRPC5
X 111187472 111187631 59M4D96M 60 TRPC5
X 111187472 111187631 59M4D96M 60 TRPC5
X 111187472 111187631 59M2D98M 60 TRPC5
X 111187472 111187631 55M6D98M 60 TRPC5
X 1

11 63176136 63176323 187M 60 SLC22A9
1
hcc38_pbs_beads_i7_H7_i5_H3_MERGED.sorted.bam
0
hcc38_pbs_beads_i7_G9_i5_G3_MERGED.sorted.bam
3 168818652 168818870 218M 60 MECOM
1
hcc38_pbs_beads_i7_G8_i5_G3_MERGED.sorted.bam
2 209113024 209113205 181M 60 IDH1
3 168818652 168818870 62M16D140M 60 MECOM
2
hcc38_pbs_beads_i7_G7_i5_G3_MERGED.sorted.bam
0
hcc38_pbs_beads_i7_F9_i5_F3_MERGED.sorted.bam
0
hcc38_pbs_beads_i7_F8_i5_F3_MERGED.sorted.bam
0
hcc38_pbs_beads_i7_E9_i5_E3_MERGED.sorted.bam
0
hcc38_pbs_beads_i7_E8_i5_E3_MERGED.sorted.bam
3 168818652 168818870 218M 60 MECOM
1
hcc38_pbs_beads_i7_E7_i5_E3_MERGED.sorted.bam
3 168818652 168818870 218M 60 MECOM
3 168818652 168818870 218M 60 MECOM
2
hcc38_pbs_beads_i7_D9_i5_D3_MERGED.sorted.bam
3 168818652 168818870 218M 60 MECOM
3 168818652 168818870 218M 60 MECOM
3 168818652 168818870 218M 60 MECOM
3 168818652 168818870 218M 60 MECOM
3 168818652 168818870 61M1I157M 60 MECOM
3 168818652 168818870 218M 60 MECOM
6
hcc38_pbs_beads_i7_D8_i5_D3_MERGED.sort

In [28]:
od=dict()
gset=set()
for k, v in counts.items():
    if not 'mgh'==k[:3]:
        total_reads=v['sum'][0]
#         print (total_reads)
        bam= k +".sorted.bam"
        ok= bam
#         print (bam)
        with pysam.AlignmentFile(bam) as sam_in:
            ttmp=dict()
#             print (sam_in.count())
            r=0
            for aread in sam_in:
                r+=1
                tmp=dict()
                chr_number=aread.reference_name
                start_p=aread.reference_start
                end_p=aread.reference_end
                vv=get_gene_name(chr_number,start_p,end_p) # v is a list of dict
                gene=vv[0]['name2']
#                 print (aread.is_unmapped, aread.is_reverse, aread.reference_name,aread.reference_start, aread.reference_end, aread.mapping_quality, gene)
#                 print (aread.reference_name,aread.reference_start, aread.reference_end,aread.get_reference_sequence())
#                 print (chr_number,start_p,end_p,aread.cigarstring, aread.mapping_quality, gene)
                tmp['chr']= aread.reference_name
                tmp['ref_start']=aread.reference_start
                tmp['ref_end']=aread.reference_end
                tmp['cigar']=aread.cigarstring
                tmp['maping_quality']=aread.mapping_quality
                tmp['annotation']=vv
                tmp['gene']=gene
                gset.add(gene)
    
                ttmp[r]=tmp
            ttmp['total_reads']=sam_in.count()

        od[ok]=ttmp
            
            
                
                
            
                
#                 print (aread.cigartuples, aread.mapping_quality, gene)
                
# 				if not aread.is_unmapped and aread.is_reverse and aread.reference_name == CHROM and aread.reference_start + 1 == POS:
# 					value = parse_cigar(aread.cigartuples)
# 					numer += value == 1
# 					denom += value < 2
# 					if value < 2:
# 						cigardict[aread.cigarstring] += 1
# 		print(cell)
# 		print(cigardict)
# 		print()        

#         s+=1
# print (s)

In [29]:
# sum_dict=dict()
d = defaultdict(lambda:defaultdict(list))

for k, v in od.items():
    for kk,vv in v.items():
        if not type(vv)==int:
            gene= vv['gene']
            d[k][gene].append(kk)


In [30]:
for k, v in d.items():
    print (k)
    for kk,vv in v.items():
        print (kk, len(vv))

hcc38_rtl_non_dig_i7_H3_i5_H11_MERGED.sorted.bam
TRAPPC12 8
IDH1 89
MECOM 12
KRAS 21
TP53BP1 32
hcc38_rtl_non_dig_i7_H2_i5_H11_MERGED.sorted.bam
MECOM 5
TP53BP1 68
hcc38_rtl_non_dig_i7_H1_i5_H11_MERGED.sorted.bam
IDH1 1
hcc38_rtl_non_dig_i7_G3_i5_G11_MERGED.sorted.bam
TRAPPC12 6
IDH1 103
MECOM 17
KRAS 1
TP53BP1 64
hcc38_rtl_non_dig_i7_G2_i5_G11_MERGED.sorted.bam
TRAPPC12 2
IDH1 112
MECOM 21
KRAS 7
TP53BP1 52
hcc38_rtl_non_dig_i7_G1_i5_G11_MERGED.sorted.bam
TRAPPC12 1
MECOM 14
KRAS 30
TP53BP1 68
hcc38_rtl_non_dig_i7_F2_i5_F11_MERGED.sorted.bam
 11
PAK3 90
hcc38_rtl_non_dig_i7_F1_i5_F11_MERGED.sorted.bam
 92
PAK3 36
TRPC5 34
hcc38_rtl_non_dig_i7_E3_i5_E11_MERGED.sorted.bam
 138
PAK3 184
TRPC5 85
hcc38_rtl_non_dig_i7_E2_i5_E11_MERGED.sorted.bam
 20
PAK3 39
TRPC5 3
hcc38_rtl_non_dig_i7_B1_i5_B11_MERGED.sorted.bam
MECOM 1
hcc38_rtl_i7_H3_i5_H2_MERGED.sorted.bam
MECOM 1
KRT1 1
hcc38_rtl_i7_G3_i5_G2_MERGED.sorted.bam
TRAPPC12 1
MECOM 1
SUSD3 1
hcc38_rtl_i7_G1_i5_G2_MERGED.sorted.bam
PLB1 1
ME

In [21]:
# Python lib for interacting with BLAT server


"""
	BLAT server info:
		gfServer start localhost 8123 canFam3.2bit &
		gfServer start localhost 8124 hg19.2bit &
		gfServer start localhost 8125 mm10.2bit &
		gfServer -trans -mask start localhost 8126 canFam3.2bit &
		gfServer -trans -mask start localhost 8127 hg19.2bit &
		gfServer -trans -mask start localhost 8128 mm10.2bit &
"""

class BlatClient:
	def __init__(self, host_name, port_number, is_protein_bool):
		self.host = host_name
		self.port = port_number
		self.is_protein = is_protein_bool

	def blat_sequence(self, nuc_string=""):
		"""
			Take a nucleotide sequence. Returns a list of BlatAlignment objects. 
		"""	
		blat_alignments = []

		to_blat = ">blatter\n%s" % nuc_string
		blat_results = os.popen("echo '%s' | /Users/ltao/biotools/blat/gfClient %s %i -minScore=0 -minIdentity=0 /Users/ltao/biotools/blat/ stdin stdout" % (to_blat ,self.host, self.port)).read().split("\n")
		for result in blat_results:
			spl = result.split("\t")
			if spl[0].isdigit():			
				algn = BlatAlignment(result,self.is_protein)
				blat_alignments.append(algn)

		return blat_alignments

class BlatAlignment:
	"""
		PSL format line and returns a BlatAlignment object
	"""
	def __init__(self, psl_line, is_protein_bool):
		matches,misMatches,repMatches,nCount,qNumInsert,qBaseInsert,tNumInsert,tBaseInsert,strand,qName,qSize,qStart,qEnd,tName,tSize,tStart,tEnd,blockCount,blockSizes,qStarts,tStarts = psl_line.split("\t")
		
		self.start = int(tStart.strip())
		self.stop = int(tEnd.strip())
		self.chromosome = tName.strip()
		self.strand = strand.strip()
		self.base_matches = int(matches.strip())
		self.mismatches = int(misMatches.strip())
		self.repeat_matches = int(repMatches.strip())
		self.query_size = int(qSize.strip())
		self.pct_match = float(self.base_matches) / float(self.query_size)
		self.span = self.stop - self.start
		self.num_gaps = int(tNumInsert.strip())
		self.total_gap_size = int(tBaseInsert.strip()) 
		self.n_masked_bases = int(nCount.strip())
		self.query_start = int(qStart.strip())
		self.query_stop = int(qEnd.strip())
		self.query_gap_count = int(qNumInsert.strip())
		self.query_gap_bases = int(qBaseInsert.strip())
		self.block_count = int(blockCount.strip())
		self.block_sizes = [int(x.strip()) for x in blockSizes.strip().split(",") if x != ""]
		self.query_block_starts = [int(x.strip()) for x in qStarts.strip().split(",") if x != ""]
		self.block_starts = [int(x.strip()) for x in tStarts.strip().split(",") if x != ""]
		self.is_protein = is_protein_bool

	def get_pct_ident(self):
		"""
			Calculates UCSCs percent identity score
		"""
		if self.is_protein:
			sizeMul = 3
		else:
			sizeMul = 1

		# Calculate alignment size
		qAliSize = sizeMul * (self.query_stop - self.query_start)
		tAliSize = self.stop - self.start
		aliSize = min(qAliSize, tAliSize)

		# Break if something fucked up
		if aliSize <= 0:
			return 0

		# Get difference in alignment size
		sizeDif = qAliSize - tAliSize
		if sizeDif < 0:
			sizeDif = -sizeDif
		
		insertFactor = self.query_gap_count

		total = (sizeMul * (self.base_matches + self.repeat_matches + self.mismatches))
		if total != 0:
			milliBad = (1000 * (self.mismatches * sizeMul + insertFactor + round(3 * math.log10(1 + sizeDif)))) / total
		
		pct_ident = 100.0 - milliBad * 0.1
		
		return pct_ident




In [53]:
counts=dict()
for mfa in mfasta:
    counts[mfa]=rank_amplicon(mfa)
    print (sum_fastq(mfa))

(10, 217, 218)
(2, 218, 243)
(19, 218, 219)
(10, 211, 218)
(17, 176, 218)
(22, 217, 227)
(19, 212, 219)
(20, 211, 218)
(9, 217, 218)
(26, 218, 219)
(17, 211, 218)
(4, 213, 218)
(18, 211, 218)
(14, 211, 218)
(5, 213, 218)
(12, 213, 218)
(13, 218, 218)
(18, 218, 219)
(16, 213, 218)
(17, 204, 218)
(9, 176, 249)
(21, 214, 218)
(11, 191, 227)
(8, 197, 218)
(27, 213, 219)
(44, 213, 227)
(31, 215, 249)
(29, 213, 219)
(30, 185, 262)
(4, 218, 218)
(22, 218, 218)
(18, 217, 218)
(1, 197, 197)
(11171, 175, 228)
(8584, 213, 228)
(5, 176, 227)
(33011, 163, 228)
(10470, 176, 228)
(11052, 175, 228)
(12088, 173, 237)
(5335, 175, 228)
(1, 218, 218)
(125, 153, 218)
(62, 152, 287)
(345, 152, 287)
(470, 152, 218)
(37, 152, 218)
(6, 213, 218)
(902, 152, 230)
(1, 286, 286)
no reads
(0, 'no reads', 'no reads')
no reads
(0, 'no reads', 'no reads')
no reads
(0, 'no reads', 'no reads')
no reads
(0, 'no reads', 'no reads')
no reads
(0, 'no reads', 'no reads')
no reads
(0, 'no reads', 'no reads')
(1, 218, 218)
(1,

In [33]:
for k in list(counts.keys()):
    print (k)
    i=counts[k]
    tmp=dict()
    for kk,ii in i.items():
        tmp[kk]=len(ii)
    from collections import OrderedDict
    dd = OrderedDict(sorted(tmp.items(), key=lambda x: x[1],reverse=True))
    counts[k]["ranking"]=dd
    for seq in list(dd.keys())[:10]:
        print (seq, dd[seq])

mgh_5_gt_i7_H4_i5_H9_MERGED
ranking 7
agaaatgtgagttccatgcaagattaattgaatcaggaataagcatctggctgctgtgtggcatgagtgaaatatagtcaactgtgctgactttctcttataaacatcttttcataacagatactgtggcaagatttttccaaggtctgcaaacctacacggcacttgagaacccacacaggagagcagccttacaggtctgacaatgatatttgct 4
agaaatgtgagttccatgcaagattaattgaatcaggaataagcatctggctgctgtgtggcatgagtgaaatatagtcaactgtgctgactttctcttataaacatcttttcataacagatactgtggcaagatttttccaaggtctgcaaacctaacacggcacttgagaacccacacaggagagcagccttacaggtctgacaatgatatttgct 2
agaaatgtgagttccatgcaagattaattgaatcaggaataagcatctggctgctgtgtggcatgagtgaaatatagtcaactgtgctgactttctcttataaacatcttttcataacagatactgtggcgagatttttccaaggtctgcaaacctaacacggcacttgagaacccacacaggagagcagccttacaggtctgacaatgatatttgct 1
agaaatgtgagttccatgcaagattaattgaatcaggaataagcatctggctgctgtgtggcatgagtgaaatatagtcaactgtgctgactttctcttataaacatcttttcataacagatactgtggcaagatttttccaaggtctgtaaacctacacggcacttgagaacccacacaggagagcagccttacaggtctgacaatgatatttgct 1
agaaatttgagttccatgcaagattaattgaatcaggaataagcatctggctgctgtgtggcatgagtgaaatatagtca

caagttggaaatttctgggccatgaaaaaaaaaaacatgcaaaatcacattattgccaacatgacttacttgatccccataagcatgacgacctatgatgataggttttacccatccactcacaagccgggggatatttttgcagataatggcttctctgaagaccgtgccacccagaatat 1
hcc38_pbs_beads_i7_H10_i5_H3_MERGED
ranking 14
agaaatgtgagttccatgcaagattaattgaatcaggaataagcatctggctgctgtgtggcatgagtgaaatatagtcaactgtgctgactttctcttataaacatcttttcataacagatactgtggcaagatttttccaaggtctgcaaacctaacacggcacttgagaacccacacaggagagcagccttacaggtctgacaatgatatttgct 9
agaaatgtgagttccatgcaagattcattgaatcaggaataagcatctggctgctgtgtggcatgagtgaaatatagtcaactgtgctgactttctcttataaacatcttttcataacagatactgtggcaagatttttccaaggtctgcaaacctaacacggcacttgagaacccacacaggagagcagccttacaggtctgacaatgatatttgct 2
agaaatgtgagttccatgcaagattaattgaatcaggaataagcatctggctgctgtgtggcatgagtgaaatatagtcaactgtgctgactttctcttataaacatcttttcataacagatactgtggcaagatttttccaaggtctgcaaacctaaacacggcacttgagaacccacacaggagagcagccttacaggtctgacaatgatatttgct 2
agaaatgtgagttccatgcaagattaattgaatcaggaataagcatctggctgctgtgtggcatgagtgaaatatagtcaactgtgctgactttctcttataaa

#  in shell run "/Users/ltao/biotools/blat/gfServer start localhost 8124 /Users/ltao/biotools/blat/hg19.2bit"

In [46]:
blatter = BlatClient('localhost',8124,False)

for k in list(counts.keys())[:10]:
    print (k)
    dd=counts[k]['ranking']
    for seq in list(dd.keys()):
#         dd['blat']=blatter.blat_sequence(seq)
        alns=blatter.blat_sequence(seq)
        tmp=dict()
        ttk=0
        for a in alns:
            ttk+=1
            ttmp=dict()
            ttmp['target']= (a.chromosome,a.start,a.stop,a.strand)
            ttmp ["Base Matches"]= a.base_matches
            ttmp["Base mismatches"]=a.mismatches
            ttmp["Percent Match"]=a.pct_match
            ttmp["Percent Identitty"]=a.get_pct_ident()
            tmp[ttk]=ttmp
        dd['alns']=tmp
#         for a in alns:
#             print (a.chromosome,a.start,a.stop,a.strand)
#             print ("Base Matches",a.base_matches)
#             print ("Base mismatches",a.mismatches)
#             print ("Percent Match",a.pct_match)
#             print ("Percent Identitty",a.get_pct_ident())
#         print ('''\n''')

i7_H9_i5_H3_S176_L001_R2_001.fastq
i7_H9_i5_H3_S176_L001_R1_001.fastq
i7_H8_i5_H3_S175_L001_R2_001.fastq


KeyboardInterrupt: 

In [58]:

counts_df=pd.DataFrame.from_dict(counts, orient='index')
counts_df.to_excel(path+'mecon_adr_counts0.1.xlsx')